In [1]:
from pathlib import Path

In [2]:
TEST_DATA_PATH = Path("test_data/the_intercept.ink")

In [3]:
import re
import sys
from collections import defaultdict, deque
from typing import Set, List, Dict, Optional, Tuple, Any, Union
import argparse
import copy
from pydantic import BaseModel, Field, validator, root_validator
from enum import Enum


In [4]:
class ChoiceType(str, Enum):
    """Enum for choice types"""
    STICKY = "+"  # Reusable choices
    REGULAR = "*"  # Consumed after use

class Choice(BaseModel):
    """Represents a choice in the Ink script"""
    text: str = Field(..., description="The display text of the choice")
    target: str = Field(..., description="The target knot name")
    condition: Optional[str] = Field(None, description="Condition for this choice to be available")
    line_number: int = Field(0, description="Line number in the source file")
    sticky: bool = Field(False, description="Whether this choice is reusable (+ choices)")
    
    @property
    def choice_type(self) -> ChoiceType:
        """Get the choice type as an enum"""
        return ChoiceType.STICKY if self.sticky else ChoiceType.REGULAR
    
    def __str__(self) -> str:
        sticky_marker = "+" if self.sticky else "*"
        condition_str = f" {{{self.condition}}}" if self.condition else ""
        return f"{sticky_marker}{condition_str} [{self.text}] -> {self.target}"
    
class VariableChange(BaseModel):
    """Represents a variable change operation"""
    change_type: str = Field(..., description="Type of change (e.g., 'assignment')")
    expression: str = Field(..., description="The variable change expression")
    
    def __str__(self) -> str:
        return f"~ {self.expression}"

class Knot(BaseModel):
    """Represents a knot (scene/section) in the Ink script"""
    name: str = Field(..., description="Name of the knot")
    content: str = Field("", description="Text content of the knot")
    choices: List[Choice] = Field(default_factory=list, description="Available choices in this knot")
    fallback_target: Optional[str] = Field(None, description="Default target if no choices are available")
    line_number: int = Field(0, description="Line number where this knot is defined")
    variable_changes: List[VariableChange] = Field(default_factory=list, description="Variable changes in this knot")
    
    @validator('name')
    def validate_name(cls, v):
        if not v or not v.strip():
            raise ValueError("Knot name cannot be empty")
        return v.strip()
    
    def get_choice_summary(self) -> Dict[str, int]:
        """Get a summary of choice types in this knot"""
        sticky_count = sum(1 for choice in self.choices if choice.sticky)
        regular_count = len(self.choices) - sticky_count
        return {"sticky": sticky_count, "regular": regular_count}
    
    def __str__(self) -> str:
        summary = self.get_choice_summary()
        fallback = f" → {self.fallback_target}" if self.fallback_target else ""
        var_changes = f" ({len(self.variable_changes)} var changes)" if self.variable_changes else ""
        return f"{self.name}: {summary['regular']} regular + {summary['sticky']} sticky choices{fallback}{var_changes}"

/var/folders/dg/t5gbyrdx6sz1cs0wlxk2957r0000gn/T/ipykernel_28433/4236243094.py:41: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('name')


In [5]:
class InkScript(BaseModel):
    """Represents a complete Ink script with all its knots and variables"""
    knots: Dict[str, Knot] = Field(default_factory=dict, description="All knots in the script")
    variables: Dict[str, Any] = Field(default_factory=dict, description="Initial variable values")
    includes: List[str] = Field(default_factory=list, description="Included files")
    
    def get_starting_knot(self, preferred_start: Optional[str] = None) -> Optional[str]:
        """Find the starting knot for path analysis"""
        if preferred_start and preferred_start in self.knots:
            return preferred_start
            
        possible_starts = [
            'start', 'scene_01_01', 'scene_1_1', 'beginning', 'intro'
        ]
        
        for start_name in possible_starts:
            if start_name in self.knots:
                return start_name
        
        # Return first knot if no standard starting point found
        return list(self.knots.keys())[0] if self.knots else None

In [6]:
class InkParser():
    """Parser for Ink script files"""
    
    
    @staticmethod
    def parse_file(filepath: str) -> InkScript:
        """Parse an Ink file and return an InkScript object."""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()
        except FileNotFoundError:
            raise FileNotFoundError(f"File '{filepath}' not found.")
        except UnicodeDecodeError:
            raise UnicodeDecodeError(f"Could not decode file '{filepath}'. Check encoding.")
            
        return InkParser.parse_content(content)
    
    @staticmethod
    def parse_content(content: str) -> InkScript:
        """Parse Ink content and extract knots, choices, and paths."""
        lines = content.split('\n')
        ink_script = InkScript()
        
        current_knot = None
        current_content = []
        line_number = 0
        
        for i, line in enumerate(lines):
            line_number = i + 1
            original_line = line
            stripped = line.strip()
            
            # Skip comments and empty lines
            if not stripped or stripped.startswith('//'):
                continue
                
            # Handle includes
            if stripped.startswith('INCLUDE'):
                include_file = stripped.split()[1]
                ink_script.includes.append(include_file)
                continue
                
            # Handle variable declarations
            if stripped.startswith('VAR '):
                var_match = re.match(r'VAR\s+(\w+)\s*=\s*(.+)', stripped)
                if var_match:
                    var_name, var_value = var_match.groups()
                    # Try to parse the value
                    try:
                        if var_value.lower() == 'true':
                            ink_script.variables[var_name] = True
                        elif var_value.lower() == 'false':
                            ink_script.variables[var_name] = False
                        elif var_value.isdigit():
                            ink_script.variables[var_name] = int(var_value)
                        elif var_value.startswith('"') and var_value.endswith('"'):
                            ink_script.variables[var_name] = var_value[1:-1]
                        else:
                            ink_script.variables[var_name] = var_value
                    except:
                        ink_script.variables[var_name] = var_value
                continue
            
            # Handle knot definitions
            if stripped.startswith('===') and stripped.endswith('==='):
                # Save previous knot if exists
                if current_knot:
                    current_knot.content = '\n'.join(current_content)
                    ink_script.knots[current_knot.name] = current_knot
                
                # Start new knot
                knot_name = stripped.strip('= ').strip()
                current_knot = Knot(
                    name=knot_name,
                    line_number=line_number
                )
                current_content = []
                continue
            
            # If we're in a knot, collect content
            if current_knot:
                # Handle variable assignments like ~ var = value or ~ var += item
                if stripped.startswith('~'):
                    var_assignment = stripped[1:].strip()
                    current_knot.variable_changes.append(
                        VariableChange(change_type='assignment', expression=var_assignment)
                    )
                    continue
                
                # Handle choices (both * and +)
                choice_match = re.match(r'([*+])\s*(\{[^}]*\})?\s*\[([^\]]*)\]\s*->\s*(\w+)', stripped)
                if choice_match:
                    choice_type, condition, choice_text, target = choice_match.groups()
                    
                    if condition:
                        condition = condition.strip('{}').strip()
                    
                    choice = Choice(
                        text=choice_text,
                        target=target,
                        condition=condition,
                        line_number=line_number,
                        sticky=(choice_type == '+')
                    )
                    current_knot.choices.append(choice)
                    continue
                
                # Handle direct redirects
                redirect_match = re.match(r'->\s*(\w+)', stripped)
                if redirect_match:
                    target = redirect_match.group(1)
                    if not current_knot.fallback_target:
                        current_knot.fallback_target = target
                    continue
                
                # Regular content
                current_content.append(original_line)
        
        # Save the last knot
        if current_knot:
            current_knot.content = '\n'.join(current_content)
            ink_script.knots[current_knot.name] = current_knot
            
        return ink_script


In [8]:
the_intercept_script = InkParser.parse_file(TEST_DATA_PATH)

In [ ]:
the_intercept_script.knots["start"].choices # this is not working properly when things are so nested...

[Choice(text='Lie', target='disagree', condition=None, line_number=154, sticky=False)]

In [13]:
the_intercept_script.knots

{'start': Knot(name='start', content='\t- \tThey are keeping me waiting. \n\t\t*\tHut 14[]. The door was locked after I sat down. \n\t\tI don\'t even have a pen to do any work. There\'s a copy of the morning\'s intercept in my pocket, but staring at the jumbled letters will only drive me mad. \n\t\tI am not a machine, whatever they say about me.\n\t- (opts)\n\t\t{|I rattle my fingers on the field table.|}\n \t\t* \t(think) [Think] \n \t\t\tThey suspect me to be a traitor. They think I stole the component from the calculating machine. They will be searching my bunk and cases. \n\t\t\tWhen they don\'t find it, {plan:then} they\'ll come back and demand I talk. \n \t\t*\t(plan) [Plan]\n \t\t\t{not think:What I am is|I am} a problem—solver. Good with figures, quick with crosswords, excellent at chess. \n \t\t\tBut in this scenario — in this trap — what is the winning play?\n \t\t\t* * \t(cooperate) [Co—operate] \n\t \t\t\t\tI must co—operate. My credibility is my main asset. To contradict m

In [14]:
# The start of a knot is indicated by two or more equals signs, as follows.

In [ ]:
# Knots can include sub-sections called "stitches". These are marked using a single equals sign.